In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [2]:
## parameters cell
itp_id = 300

In [3]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [4]:
analysis_date = shared_utils.rt_dates.DATES['aug2024']

In [5]:
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [6]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
38,City of Torrance,339,recvzE9NXgGMmqcTH,07 - Los Angeles,True,2024-08-14,7cb3592df114dab1288176f1410563a1,speedmap_segs_available


In [7]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [8]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [9]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [10]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry'],
      dtype='object')

In [11]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [12]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
214101,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,offpeak,3.69,10,1.87,5.91,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."
214102,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,peak,4.33,14,3.26,7.16,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."
214103,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,all_day,4.22,24,2.91,7.16,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."


In [13]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [14]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [15]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [16]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [17]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [18]:
total_peak

8

In [19]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [20]:
peak_test = prep_gdf(peak_test)

In [21]:
cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

##  adding identifiers for map display

In [24]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,trips_per_hour
214296,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,297__298,Pacific Av at 10th St__PACIFIC AV + 7TH ST,297-298-1,peak,12.67,1,12.67,12.67,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"POLYGON ((167314.431 -469149.031, 167315.398 -...",0.125
214298,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,298__299,PACIFIC AV + 7TH ST__Long Beach Bl at 6th St,298-299-1,peak,12.56,1,12.56,12.56,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"POLYGON ((167316.041 -469554.175, 167317.093 -...",0.125


In [25]:
gpd.read_file('300_AM_Peak_speeds.geojson') >> head(3)

,id,stop_id,stop_name,shape_id,stop_sequence,route_id,route_short_name,direction_id,p50_mph,p20_mph,p80_mph,fast_slow_ratio,trips_per_hour,miles_from_last,time_formatted,organization_name,geometry
0,306,1129,MAIN NB & OLYMPIC FS,26731,2.0,3709,5,0.0,3.4,3.4,3.4,1.0,0.3,0.1,1:45,City of Santa Monica,"POLYGON ((-118.49075 34.01082, -118.49099 34.0..."
1,307,292,2ND NB & BROADWAY NS,26731,3.0,3709,5,0.0,9.2,9.2,9.2,1.0,0.3,0.3,1:57,City of Santa Monica,"POLYGON ((-118.49316 34.01241, -118.49383 34.0..."
2,308,104,SANTA MONICA EB & 3RD FS,26731,4.0,3709,5,0.0,13.0,13.0,13.0,1.0,0.3,0.2,0:55,City of Santa Monica,"POLYGON ((-118.49673 34.01480, -118.49673 34.0..."


##  to-do

* where does new n_trips come from? should be schedule-based for speedmaps applications...
* split am/pm peak
* get route names from here? https://github.com/cal-itp/data-analyses/blob/main/gtfs_funnel/clean_route_naming.py

In [26]:
import sys

In [27]:
sys.path.append('../gtfs_funnel/')

In [28]:
from update_vars import GTFS_DATA_DICT, SCHED_GCS, all_dates

In [30]:
CLEANED_ROUTE_NAMING = GTFS_DATA_DICT.schedule_tables.route_identification

In [39]:
analysis_dt = dt.datetime.fromisoformat(analysis_date)

In [42]:
route_df = pd.read_parquet(f"{SCHED_GCS}{CLEANED_ROUTE_NAMING}.parquet",
                           filters=[('service_date', '==', analysis_dt)],
                           columns=['schedule_gtfs_dataset_key', 'route_id', 'route_short_name', 'service_date'])

In [43]:
route_df >> head(3)

,schedule_gtfs_dataset_key,route_id,route_short_name,service_date
17001,1770249a5a2e770ca90628434d4934b1,3407,Route 18,2024-08-14
17002,1770249a5a2e770ca90628434d4934b1,3408,Route 21,2024-08-14
17003,1770249a5a2e770ca90628434d4934b1,3398,Route 7,2024-08-14


In [36]:
# route_df >> distinct(_.service_date) >> arrange(-_.service_date)

## export map

In [22]:
speedmap_state = shared_utils.rt_utils.set_state_export(peak_test, filename='testnew6', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew6.geojson.gz


In [23]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-08-14',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew6.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (33.85596962015502, -118.31386191576537),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDgtMTQiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdG5ldzYuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzMuODU1OTY5NjIwMTU1MDIsIC0xMTguMzEzODYxOTE1NzY1MzddLCAiem9vbSI6IDEzLCAibGVnZW5kX3V